In [ ]:
import time
import odrive
from odrive.enums import *

print("Finding odrv...")
odrv0 = odrive.find_any()

def CALIB():
    print("starting calibration...")
    time.sleep(1)
    odrv0.axis0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    odrv0.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    while odrv0.axis0.current_state != AXIS_STATE_IDLE:
        time.sleep(0.1)
    while odrv0.axis1.current_state != AXIS_STATE_IDLE:
        time.sleep(0.1)

    print("setting params axis 0...")
    odrv0.axis0.encoder.set_linear_count(0)
    odrv0.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

    print("setting params axis 1...")
    odrv0.axis1.encoder.set_linear_count(0)
    odrv0.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
def STOP():
    odrv0.axis0.requested_state = AXIS_STATE_IDLE
    odrv0.axis1.requested_state = AXIS_STATE_IDLE
def rezero(axis, dir): # 1 = CCW, -1 = CW <-- dir indicates which direction you wish it so spin in while in use
    if axis == 0:
        odrv0.axis0.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
        odrv0.axis0.controller.config.input_mode = INPUT_MODE_PASSTHROUGH
    if axis == 1:
        odrv0.axis1.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
        odrv0.axis1.controller.config.input_mode = INPUT_MODE_PASSTHROUGH

    if axis == 0:
        init = odrv0.axis0.controller.input_pos
        print(init)
        while dir * init > 0:
            odrv0.axis0.controller.input_pos = init
            init += -1 * dir * 0.01
            time.sleep(0.01)
    elif axis == 1:
        init = odrv0.axis1.controller.input_pos
        print(init)
        while dir * init > 0:
            odrv0.axis1.controller.input_pos = init
            init += -1 * dir * 0.01
            time.sleep(0.01)
rezero(axis=0, dir=-1)
rezero(axis=1, dir=1)

import math
L1 = 275
L2 = 275
def q1q2_from_xy(x, y):
    xD_mag = math.sqrt(x*x + y*y)
    c2 = (xD_mag * xD_mag - L1 * L1 - L2 * L2) / (2 * L1 * L2)
    if abs(c2) > 1:
        return None
    if c2 == 1:
        return math.atan2(y, x), 0, math.atan2(y, x), 0
    q2_1 = 1 * math.acos( c2 )
    q2_2 = -1 * math.acos( c2 )
    theta = math.atan2(y, x)
    q1_1 = theta - math.atan2(L2 * math.sin(q2_1), L1 + L2 * math.cos(q2_1))
    q1_2 = theta - math.atan2(L2 * math.sin(q2_2), L1 + L2 * math.cos(q2_2))
    return q1_1, q2_1, q1_2, q2_2

CALIB()


In [ ]:
import time
import math

time.sleep(3)

odrv0.axis0.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
odrv0.axis0.controller.config.input_mode = INPUT_MODE_PASSTHROUGH
odrv0.axis1.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
odrv0.axis1.controller.config.input_mode = INPUT_MODE_PASSTHROUGH

final_standing_height = 300

def func1(t):
    y = -100 * math.pi * t + 50 * math.pi
    x = 500 - 100 * math.cos(0.01 * y) 
    return x,y

def func2(t):
    y = 100 * math.pi * t - 150 * math.pi
    x = 500
    return x,y
N = 5
for i in range(N):
    t_ = 0
    # Path type 1
    while t_ < 1:
        x, y = func1(t_)
        q1_1, q2_1, q1_2, q2_2 = 0, 0, 0, 0
        q1_1, q2_1, q1_2, q2_2 = q1q2_from_xy(x, y)
        odrv0.axis1.controller.input_pos = q1_2 * (10 / (2 * math.pi))
        odrv0.axis0.controller.input_pos = q2_2 * (10 / (2 * math.pi))
        t_ += 0.001
        time.sleep(0.001)

    time.sleep(0.1)

    # Path type 2
    while t_ < 2:
        x, y = func2(t_)
        q1_1, q2_1, q1_2, q2_2 = q1q2_from_xy(x, y)
        odrv0.axis1.controller.input_pos = q1_2 * (10 / (2 * math.pi))
        odrv0.axis0.controller.input_pos = q2_2 * (10 / (2 * math.pi))
        t_ += 0.001
        time.sleep(0.001)